# 0.赛题链接

锂离子电池生产参数调控及生产温度预测挑战赛：


>  [报名点这里](https://challenge.xfyun.cn/topic/info?type=lithium-ion-battery&ch=ymfk4uU)
  

本项目在普通CPU环境所需时长大概50分钟,V100 32G环境大概38分钟,A100环境大概10分钟

# 1. 赛事报名
1. 点击报名参赛，登录讯飞开放平台。（注意勾选自动注册）

<img src="https://ai-studio-static-online.cdn.bcebos.com/32ee2ae3384b45a2a63311fe2acc628fbf4ec9c14f6942679455e8fbc6f7faac" width="75%" />

<img src="https://ai-studio-static-online.cdn.bcebos.com/0473562fa8984c7c81014c0ffb7474f4a4e1b65c40f9493cb03c1d842d317572" width="75%" />

2. 看到恭喜！您已成功报名的提示后，表明报名完成。

<img src="https://ai-studio-static-online.cdn.bcebos.com/41d19a29f0d241eb8325a95d49116fc1caacec49fcb34dc5996efa968f521c9b" width="75%" />

<img src="https://ai-studio-static-online.cdn.bcebos.com/7d4bb7ad707342b1af7924aba9fc781a81edac3438c94c8cb2784a0e7ab3dabc" width="75%" />

3. 基线运行后得出的结果可以在提交结果这里提交, 快来运行试试吧~

<img src="https://ai-studio-static-online.cdn.bcebos.com/28b50d1b6fa248ca9c8f057778cffa8deba01694e1d54de4a0b47e97a902800c" width="75%" />

<img src="https://ai-studio-static-online.cdn.bcebos.com/fb50886e63a14c90ba97787e3ff79845013fc2675643451dae472b01677b5c15" width="75%" />


# 2. 赛题介绍

## 2.1 任务要求

- 初赛任务：初赛提供了电炉17个温区的实际生产数据，分别是电炉上部17组加热棒设定温度T1-1 ~ T1-17，电炉下部17组加热棒设定温度T2-1~T2-17，底部17组进气口的设定进气流量V1-V17，选手需要根据提供的数据样本构建模型，预测电炉上下部空间17个测温点的测量温度值。

## 2.2 数据集介绍

- 初赛任务：初赛提供了电炉17个温区的实际生产数据，分别是电炉上部17组加热棒设定温度T1-1 ~ T1-17，电炉下部17组加热棒设定温度T2-1~T2-17，底部17组进气口的设定进气流量V1-V17，选手需要根据提供的数据样本构建模型，预测电炉上下部空间17个测温点的测量温度值。

## 2.3 评估指标

- 初赛考核办法采用测试集各行数据的加热棒上部温度设定值、加热棒下部温度设定值、进气流量3类数据作为输入，选手分别预测上部空间测量温度、下部空间测量温度。将选手预测的上部空间测量温度、下部空间测量温度与测试集数据的测量值进行比较。采用MAE平均绝对误差作为评价指标。


# 3. 赛题分析

本次比赛为数据挖掘类型的比赛，聚焦于工业场景。本赛题实质上为回归任务，其中会涉及到时序预测相关的知识。

通过电炉空间温度推测产品内部温度，设计烧结过程的温度场和浓度场的最优控制律：
- 任务输入：电炉对应17个温区的实际生产数据，分别是电炉上部17组加热棒设定温度T1-1 ~ T1-17，电炉下部17组加热棒设定温度T2-1~T2-17，底部17组进气口的设定进气流量V1-V17；
- 任务输出：电炉对应17个温区上部空间和下部空间17个测温点的测量温度值。

值得注意的是预测目标为**34**个，所以需要我们进行34次模型训练和预测。

同时数据规模比较小，可以快速处理数据和搭建模型，对于机器要求8g内存即可。

本次为结构化赛题，包含电炉烧结每个时间段的流量、上下部设定温度，以及预测目标上下部测量温度值。


# 4. 基于LightGBM模型

在处理这个问题时，我们主要考虑的是回归预测。一种常规的解决思路是运用机器学习技术，例如 LightGBM 或 XGBoost，或者借助深度学习方法进行实践。当我们选择自行搭建模型的路径时，我们将面临更为复杂的挑战，包括构建模型结构以及对数值数据进行标准化处理。

然而，一个简易的解决方案可能就在我们眼前，那就是直接使用现成的机器学习模型。这种方法具有明显的优势，其模型使用简单，数据预处理的需求也大大减少。

总的来说，我们需要经过以下步骤来解决本问题：

- 数据预处理
- 切分训练集与验证集
- 训练模型
- 生成最后的预测结果。


在实施这些步骤的过程中，我们需要根据模型的性质和数据的特点灵活调整，确保每一步的实施都能最大化模型的预测准确性，从而有效解决这个回归预测问题。

In [2]:
# 更新lightGBM库&解压缩数据
# 注: 只需运行该行一次，若后续想要run all, 需要注释掉 !unzip data/data227148/data.zip, python的注释符号为 #
!pip install -U lightgbm
!unzip data/data227148/data.zip

Looking in indexes: https://mirrors.cernet.edu.cn/pypi/web/simple
Archive:  data/data227148/data.zip
  inflating: data/test.csv           
  inflating: data/train.csv          


In [2]:
# 导入所需要的库
import pandas as pd # 用于处理数据的工具
import lightgbm as lgb # 机器学习模型 LightGBM
from sklearn.metrics import mean_absolute_error # 评分 MAE 的计算函数
from sklearn.model_selection import train_test_split # 拆分训练集与验证集工具
from tqdm import tqdm # 显示循环的进度条工具
import warnings
warnings.filterwarnings("ignore")

In [4]:
# 数据准备
train_dataset = pd.read_csv("./data/train.csv") # 原始训练数据。
test_dataset = pd.read_csv("./data/test.csv") # 原始测试数据（用于提交）。

submit = pd.DataFrame() # 定义提交的最终数据。
submit["序号"] = test_dataset["序号"] # 对齐测试数据的序号。

MAE_scores = dict() # 定义评分项。

In [5]:
# 参数设置
pred_labels = list(train_dataset.columns[-34:]) # 需要预测的标签。
train_set, valid_set = train_test_split(train_dataset, test_size=0.2) # 拆分数据集。

# 设定 LightGBM 训练参，查阅参数意义：https://lightgbm.readthedocs.io/en/latest/Parameters.html
lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'mae',
        'min_child_weight': 5,
        'num_leaves': 2 ** 5,
        'lambda_l2': 10,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 4,
        'learning_rate': 0.05,
        'seed': 2023,
        'nthread' : 16,
        'verbose' : -1,
    }

no_info = lgb.callback.log_evaluation(period=-1) # 禁用训练日志输出。

In [6]:
# 时间特征函数
def time_feature(data: pd.DataFrame, pred_labels: list=None) -> pd.DataFrame:
    """提取数据中的时间特征。

    输入: 
        data: Pandas.DataFrame
            需要提取时间特征的数据。

        pred_labels: list, 默认值: None
            需要预测的标签的列表。如果是测试集，不需要填入。
    
    输出: data: Pandas.DataFrame
            提取时间特征后的数据。
    """
    
    data = data.copy() # 复制数据，避免后续影响原始数据。
    data = data.drop(columns=["序号"]) # 去掉”序号“特征。
    
    data["时间"] = pd.to_datetime(data["时间"]) # 将”时间“特征的文本内容转换为 Pandas 可处理的格式。
    data["month"] = data["时间"].dt.month # 添加新特征“month”，代表”当前月份“。
    data["day"] = data["时间"].dt.day # 添加新特征“day”，代表”当前日期“。
    data["hour"] = data["时间"].dt.hour # 添加新特征“hour”，代表”当前小时“。
    data["minute"] = data["时间"].dt.minute # 添加新特征“minute”，代表”当前分钟“。
    data["weekofyear"] = data["时间"].dt.isocalendar().week.astype(int) # 添加新特征“weekofyear”，代表”当年第几周“，并转换成 int，否则 LightGBM 无法处理。
    data["dayofyear"] = data["时间"].dt.dayofyear # 添加新特征“dayofyear”，代表”当年第几日“。
    data["dayofweek"] = data["时间"].dt.dayofweek # 添加新特征“dayofweek”，代表”当周第几日“。
    data["is_weekend"] = data["时间"].dt.dayofweek // 6 # 添加新特征“is_weekend”，代表”是否是周末“，1 代表是周末，0 代表不是周末。

    data = data.drop(columns=["时间"]) # LightGBM 无法处理这个特征，它已体现在其他特征中，故丢弃。

    if pred_labels: # 如果提供了 pred_labels 参数，则执行该代码块。
        data = data.drop(columns=[*pred_labels]) # 去掉所有待预测的标签。
    
    return data # 返回最后处理的数据。

test_features = time_feature(test_dataset) # 处理测试集的时间特征，无需 pred_labels。
test_features.head(5)

,流量1,流量2,流量3,流量4,流量5,流量6,流量7,流量8,流量9,流量10,...,下部温度设定16,下部温度设定17,month,day,hour,minute,weekofyear,dayofyear,dayofweek,is_weekend
0,24.222000,24.070000,28.889000,25.264999,26.827999,4.354,21.784,24.323000,16.721001,18.652000,...,837,750,3,1,4,15,9,60,2,0
1,24.138000,23.974001,28.798000,25.517000,27.108999,4.377,21.886,24.384001,16.709999,18.600000,...,837,750,3,1,4,20,9,60,2,0
2,24.152000,24.006001,28.827999,25.114000,26.719999,4.293,21.611,24.099001,16.591999,18.471001,...,837,750,3,1,4,26,9,60,2,0
3,24.108999,23.974001,28.783001,25.114000,27.047001,4.354,21.857,24.337999,16.677999,18.561001,...,837,750,3,1,4,31,9,60,2,0
4,24.350000,24.215000,29.146999,25.416000,25.416000,4.377,21.900,24.444000,16.775999,18.704000,...,837,750,3,1,4,36,9,60,2,0


In [7]:
# 从所有待预测特征中依次取出标签进行训练与预测。
for pred_label in tqdm(pred_labels):
    # print("当前的pred_label是：", pred_label)
    train_features = time_feature(train_set, pred_labels=pred_labels) # 处理训练集的时间特征。
    # train_features = enhancement(train_features_raw)
    train_labels = train_set[pred_label] # 训练集的标签数据。
    # print("当前的train_labels是：", train_labels)
    train_data = lgb.Dataset(train_features, label=train_labels) # 将训练集转换为 LightGBM 可处理的类型。

    valid_features = time_feature(valid_set, pred_labels=pred_labels) # 处理验证集的时间特征。
    # valid_features = enhancement(valid_features_raw)
    valid_labels = valid_set[pred_label] # 验证集的标签数据。
    # print("当前的valid_labels是：", valid_labels)
    valid_data = lgb.Dataset(valid_features, label=valid_labels) # 将验证集转换为 LightGBM 可处理的类型。

    # 训练模型，参数依次为：导入模型设定参数、导入训练集、设定模型迭代次数（200）、导入验证集、禁止输出日志
    model = lgb.train(lgb_params, train_data, 200, valid_sets=valid_data, callbacks=[no_info])

    valid_pred = model.predict(valid_features, num_iteration=model.best_iteration) # 选择效果最好的模型进行验证集预测。
    test_pred = model.predict(test_features, num_iteration=model.best_iteration) # 选择效果最好的模型进行测试集预测。
    MAE_score = mean_absolute_error(valid_pred, valid_labels) # 计算验证集预测数据与真实数据的 MAE。
    MAE_scores[pred_label] = MAE_score # 将对应标签的 MAE 值 存入评分项中。

    submit[pred_label] = test_pred # 将测试集预测数据存入最终提交数据中。
     
submit.to_csv('submit_result.csv', index=False) # 保存最后的预测结果到 submit_result.csv

100%|██████████| 34/34 [10:00<00:00, 17.67s/it]


> 运行效果展示:(可自行利用matplotlib绘制损失图像)

![](https://ai-studio-static-online.cdn.bcebos.com/acbf1558bed74aaeb2e5011c574a49079ba4afd9edbf424ebd2d2a69ecf6824d)


In [8]:
# 保存文件并查看结果
submit.to_csv('submit_result.csv', index=False) # 保存最后的预测结果到 submit_result.csv。
print(MAE_scores) # 查看各项的 MAE 值。

{'上部温度1': 2.905630023070507, '上部温度2': 2.8415226931217616, '上部温度3': 2.1361846091427057, '上部温度4': 1.4496459962586636, '上部温度5': 1.68903085392773, '上部温度6': 1.6314435398334883, '上部温度7': 1.0781948183757775, '上部温度8': 0.16096641603080394, '上部温度9': 0.09897319830448861, '上部温度10': 0.08914833881720488, '上部温度11': 0.10187757164033058, '上部温度12': 0.12292278944665096, '上部温度13': 0.12126945833448521, '上部温度14': 0.1766633700681394, '上部温度15': 0.29104872967795686, '上部温度16': 0.3437734306652143, '上部温度17': 0.85861714159532, '下部温度1': 2.2606565123722797, '下部温度2': 1.3787310999676785, '下部温度3': 0.7124649308763438, '下部温度4': 0.4281160357598973, '下部温度5': 0.7243745420095603, '下部温度6': 0.5348889842325286, '下部温度7': 0.3069986464042002, '下部温度8': 0.1089651260075026, '下部温度9': 16.759706644902828, '下部温度10': 0.09673220188836894, '下部温度11': 0.14445022656469653, '下部温度12': 0.16426707048885453, '下部温度13': 0.14258925662085908, '下部温度14': 0.15326240331042193, '下部温度15': 0.25333321582243373, '下部温度16': 0.3182807359451992, '下部温度17': 0.4015241

当这个Baseline跑完以后大概能取得的成绩是:

![](https://ai-studio-static-online.cdn.bcebos.com/f464f7c95a764944b106cdaa6e856bead4baeda9539f4aa0a1012e10afd6af28)

# 5. 进阶实战

在实践尝试阶段，我们使用lightgbm完成了基本的模型训练，已经最终结果输出。在进阶实践部分，将在原有Baseline基础上做进阶代码，进阶版本代码。具体地，我们可以尝试提取更多特征改善最终结果，这也是数据挖掘比赛中的主要优化方向，很多情况下决定着最终的成绩。

这里主要构建了交叉特征、历史平移特征、差分特征、和窗口统计特征；每种特征都是有理可据的，具体说明如下：

**（1）交叉特征**：主要提取流量、上部温度设定、下部温度设定之间的关系；

**（2）历史平移特征**：通过历史平移获取上个阶段的信息；

**（3）差分特征**：可以帮助获取相邻阶段的增长差异，描述数据的涨减变化情况。在此基础上还可以构建相邻数据比值变化、二阶差分等；

**（4）窗口统计特征**：窗口统计可以构建不同的窗口大小，然后基于窗口范围进统计均值、最大值、最小值、中位数、方差的信息，可以反映最近阶段数据的变化情况。

## 5.1 新特征展示

In [ ]:
# 新加入特征代码展示(不必运行)

# 交叉特征
for i in range(1,18):
    train[f'流量{i}/上部温度设定{i}'] = train[f'流量{i}'] / train[f'上部温度设定{i}']
    test[f'流量{i}/上部温度设定{i}'] = test[f'流量{i}'] / test[f'上部温度设定{i}']
    
    train[f'流量{i}/下部温度设定{i}'] = train[f'流量{i}'] / train[f'下部温度设定{i}']
    test[f'流量{i}/下部温度设定{i}'] = test[f'流量{i}'] / test[f'下部温度设定{i}']
    
    train[f'上部温度设定{i}/下部温度设定{i}'] = train[f'上部温度设定{i}'] / train[f'下部温度设定{i}']
    test[f'上部温度设定{i}/下部温度设定{i}'] = test[f'上部温度设定{i}'] / test[f'下部温度设定{i}']
    
# 历史平移
for i in range(1,18):
    train[f'last1_流量{i}'] = train[f'流量{i}'].shift(1)
    train[f'last1_上部温度设定{i}'] = train[f'上部温度设定{i}'].shift(1)
    train[f'last1_下部温度设定{i}'] = train[f'下部温度设定{i}'].shift(1)
    
    test[f'last1_流量{i}'] = test[f'流量{i}'].shift(1)
    test[f'last1_上部温度设定{i}'] = test[f'上部温度设定{i}'].shift(1)
    test[f'last1_下部温度设定{i}'] = test[f'下部温度设定{i}'].shift(1)

# 差分特征
for i in range(1,18):
    train[f'last1_diff_流量{i}'] = train[f'流量{i}'].diff(1)
    train[f'last1_diff_上部温度设定{i}'] = train[f'上部温度设定{i}'].diff(1)
    train[f'last1_diff_下部温度设定{i}'] = train[f'下部温度设定{i}'].diff(1)
    
    test[f'last1_diff_流量{i}'] = test[f'流量{i}'].diff(1)
    test[f'last1_diff_上部温度设定{i}'] = test[f'上部温度设定{i}'].diff(1)
    test[f'last1_diff_下部温度设定{i}'] = test[f'下部温度设定{i}'].diff(1)
    
# 窗口统计
for i in range(1,18):
    train[f'win3_mean_流量{i}'] = (train[f'流量{i}'].shift(1) + train[f'流量{i}'].shift(2) + train[f'流量{i}'].shift(3)) / 3
    train[f'win3_mean_上部温度设定{i}'] = (train[f'上部温度设定{i}'].shift(1) + train[f'上部温度设定{i}'].shift(2) + train[f'上部温度设定{i}'].shift(3)) / 3
    train[f'win3_mean_下部温度设定{i}'] = (train[f'下部温度设定{i}'].shift(1) + train[f'下部温度设定{i}'].shift(2) + train[f'下部温度设定{i}'].shift(3)) / 3
    
    test[f'win3_mean_流量{i}'] = (test[f'流量{i}'].shift(1) + test[f'流量{i}'].shift(2) + test[f'流量{i}'].shift(3)) / 3
    test[f'win3_mean_上部温度设定{i}'] = (test[f'上部温度设定{i}'].shift(1) + test[f'上部温度设定{i}'].shift(2) + test[f'上部温度设定{i}'].shift(3)) / 3
    test[f'win3_mean_下部温度设定{i}'] = (test[f'下部温度设定{i}'].shift(1) + test[f'下部温度设定{i}'].shift(2) + test[f'下部温度设定{i}'].shift(3)) / 3

## 5.2 优化后完整代码展示

In [3]:
# 加入新特征后的完整代码如下：（对每行代码配置解析）

# 导入所需的库
import pandas as pd # 用于处理数据的工具
import lightgbm as lgb # 机器学习模型 LightGBM
from sklearn.metrics import mean_absolute_error # 评分 MAE 的计算函数
from sklearn.model_selection import train_test_split # 拆分训练集与验证集工具
from tqdm import tqdm # 显示循环的进度条工具

# 数据准备
train_dataset = pd.read_csv("./data/train.csv") # 原始训练数据。
test_dataset = pd.read_csv("./data/test.csv") # 原始测试数据（用于提交）。

submit = pd.DataFrame() # 定义提交的最终数据。
submit["序号"] = test_dataset["序号"] # 对齐测试数据的序号。

MAE_scores = dict() # 定义评分项。

# 模型训练
pred_labels = list(train_dataset.columns[-34:]) # 需要预测的标签。
train_set, valid_set = train_test_split(train_dataset, test_size=0.2) # 拆分数据集。

# 设定 LightGBM 训练参，查阅参数意义：https://lightgbm.readthedocs.io/en/latest/Parameters.html
lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'mae',
        'min_child_weight': 5,
        'num_leaves': 2 ** 5,
        'lambda_l2': 10,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 4,
        'learning_rate': 0.05,
        'seed': 2023,
        'nthread' : 16,
        'verbose' : -1,
    }

no_info = lgb.callback.log_evaluation(period=-1) # 禁用训练日志输出。


def data_feature(data: pd.DataFrame, pred_labels: list=None) -> pd.DataFrame:
    
    data = data.copy() # 复制数据，避免后续影响原始数据。
    data = data.drop(columns=["序号"]) # 去掉”序号“特征。
    
    data["时间"] = pd.to_datetime(data["时间"]) # 将”时间“特征的文本内容转换为 Pandas 可处理的格式。
    data["month"] = data["时间"].dt.month # 添加新特征“month”，代表”当前月份“。
    data["day"] = data["时间"].dt.day # 添加新特征“day”，代表”当前日期“。
    data["hour"] = data["时间"].dt.hour # 添加新特征“hour”，代表”当前小时“。
    data["minute"] = data["时间"].dt.minute # 添加新特征“minute”，代表”当前分钟“。
    data["weekofyear"] = data["时间"].dt.isocalendar().week.astype(int) # 添加新特征“weekofyear”，代表”当年第几周“，并转换成 int，否则 LightGBM 无法处理。
    data["dayofyear"] = data["时间"].dt.dayofyear # 添加新特征“dayofyear”，代表”当年第几日“。
    data["dayofweek"] = data["时间"].dt.dayofweek # 添加新特征“dayofweek”，代表”当周第几日“。
    data["is_weekend"] = data["时间"].dt.dayofweek // 6 # 添加新特征“is_weekend”，代表”是否是周末“，1 代表是周末，0 代表不是周末。

    data = data.drop(columns=["时间"]) # LightGBM 无法处理这个特征，它已体现在其他特征中，故丢弃。

    # 交叉特征
    for i in range(1,18):
        data[f'流量{i}/上部温度设定{i}'] = data[f'流量{i}'] / data[f'上部温度设定{i}']   
        data[f'流量{i}/下部温度设定{i}'] = data[f'流量{i}'] / data[f'下部温度设定{i}']
        data[f'上部温度设定{i}/下部温度设定{i}'] = data[f'上部温度设定{i}'] / data[f'下部温度设定{i}']
        
    # 历史平移
        data[f'last1_流量{i}'] = data[f'流量{i}'].shift(1)
        data[f'last1_上部温度设定{i}'] = data[f'上部温度设定{i}'].shift(1)
        data[f'last1_下部温度设定{i}'] = data[f'下部温度设定{i}'].shift(1)

    # 差分特征
        data[f'last1_diff_流量{i}'] = data[f'流量{i}'].diff(1)
        data[f'last1_diff_上部温度设定{i}'] = data[f'上部温度设定{i}'].diff(1)
        data[f'last1_diff_下部温度设定{i}'] = data[f'下部温度设定{i}'].diff(1)
        
    # 窗口统计
    for i in range(1,18):
        data[f'win3_mean_流量{i}'] = (data[f'流量{i}'].shift(1) + data[f'流量{i}'].shift(2) + data[f'流量{i}'].shift(3)) / 3
        data[f'win3_mean_上部温度设定{i}'] = (data[f'上部温度设定{i}'].shift(1) + data[f'上部温度设定{i}'].shift(2) + data[f'上部温度设定{i}'].shift(3)) / 3
        data[f'win3_mean_下部温度设定{i}'] = (data[f'下部温度设定{i}'].shift(1) + data[f'下部温度设定{i}'].shift(2) + data[f'下部温度设定{i}'].shift(3)) / 3

    if pred_labels: # 如果提供了 pred_labels 参数，则执行该代码块。
        data = data.drop(columns=[*pred_labels]) # 去掉所有待预测的标签。
    
    return data # 返回最后处理的数据。

test_features = data_feature(test_dataset) # 处理测试集的时间特征，无需 pred_labels。


# 从所有待预测特征中依次取出标签进行训练与预测。
for pred_label in tqdm(pred_labels):
    train_features = data_feature(train_set, pred_labels=pred_labels) # 处理训练集的时间特征。
    train_labels = train_set[pred_label] # 训练集的标签数据。
    train_data = lgb.Dataset(train_features, label=train_labels) # 将训练集转换为 LightGBM 可处理的类型。

    valid_features = data_feature(valid_set, pred_labels=pred_labels) # 处理验证集的时间特征。
    valid_labels = valid_set[pred_label] # 验证集的标签数据。
    valid_data = lgb.Dataset(valid_features, label=valid_labels) # 将验证集转换为 LightGBM 可处理的类型。

    # 训练模型，参数依次为：导入模型设定参数、导入训练集、设定模型迭代次数（200）、导入验证集、禁止输出日志
    model = lgb.train(lgb_params, train_data, 200, valid_sets=valid_data, callbacks=[no_info])

    valid_pred = model.predict(valid_features, num_iteration=model.best_iteration) # 选择效果最好的模型进行验证集预测。
    test_pred = model.predict(test_features, num_iteration=model.best_iteration) # 选择效果最好的模型进行测试集预测。
    MAE_score = mean_absolute_error(valid_pred, valid_labels) # 计算验证集预测数据与真实数据的 MAE。
    MAE_scores[pred_label] = MAE_score # 将对应标签的 MAE 值 存入评分项中。

    submit[pred_label] = test_pred # 将测试集预测数据存入最终提交数据中。
     
submit.to_csv('submit_result_new.csv', index=False) # 保存最后的预测结果到 submit_result_new.csv。
print(MAE_scores) # 查看各项的 MAE 值。

100%|██████████| 34/34 [01:29<00:00,  2.64s/it]


{'上部温度1': 5.81640323792841, '上部温度2': 5.379476883082123, '上部温度3': 3.808399748500084, '上部温度4': 2.272266644361848, '上部温度5': 3.0196864642109738, '上部温度6': 2.4409575993235064, '上部温度7': 1.6268359508933687, '上部温度8': 0.13403363266770907, '上部温度9': 0.06506776580997325, '上部温度10': 0.06310893354376279, '上部温度11': 0.08672301559038695, '上部温度12': 0.11052733475383708, '上部温度13': 0.09850835812880102, '上部温度14': 0.12484363830475316, '上部温度15': 0.22184738371862178, '上部温度16': 0.33413596797800255, '上部温度17': 0.9668613851306717, '下部温度1': 3.5121356114040965, '下部温度2': 1.6087510285785738, '下部温度3': 0.707367622193821, '下部温度4': 0.3414842323560205, '下部温度5': 1.1523313711004197, '下部温度6': 0.7187381388126082, '下部温度7': 0.3106835623698695, '下部温度8': 0.06263489919093326, '下部温度9': 4.770147707473743, '下部温度10': 0.060885257246776, '下部温度11': 0.11211540002259171, '下部温度12': 0.11094016133485726, '下部温度13': 0.09245618344272785, '下部温度14': 0.09900800862479528, '下部温度15': 0.16961688575401582, '下部温度16': 0.25893549356853496, '下部温度17': 0.2874090

当这个优化措施做了能够达到的成绩大概是:

![](https://ai-studio-static-online.cdn.bcebos.com/fae80c2eea3b406ab284983a47bf8a87e6153b8935f24ceaadcd41d5f4eb7f9d)

可以看出仍然是远远不够的，因此接下来给予一些上分建议

# 6. 结语

## 6.1 优化建议

1、过拟合现象

在代码运行的时候可以将`callbacks=[no_info]`去除，以查看模型运行的日志，运行结束后也可以利用代码`print(MAE_scores)`来打印出最终的损失，观察不难看出，模型会在一定时间后进入过拟合，因此可以尝试以一些措施来缓解这种现象。

- 利用lightgbm自带的early_stopping_rounds参数。
- 采用多折验证的方式扩大数据量，使得模型泛化能力增强。


2、 参数设定

此次基线没有做精确的调参处理，因此可以调参的范围还是挺大的，常用的搜索参数策略有两种:

- 网格搜索（Grid Search）：这是一种传统的参数调整方法，它会测试指定参数的所有可能组合来找出最佳参数。但是，当参数空间较大时，这种方法可能会消耗大量计算资源和时间。

- 随机搜索（Random Search）：与网格搜索相比，随机搜索不会测试所有的参数组合，而是在参数空间中随机选择一定数量的参数组合进行测试。尽管随机搜索可能无法找到最优的参数组合，但在计算资源有限的情况下，它是一个有效的选择。

3、 模型设定

我们此次基线采用的是LightGBM，我们也可以试试XGBoost, Adaboost， Catboost等等传统的模型，并且也可以使用深度学习的方法构建循环神经网络来处理此次任务，因为本质是一个时序任务，在采用机器学习的同时也可以保存模型的参数配置，尝试进行模型集成的训练。

4、 特征

在特征选择方面，目前给出的特征仅仅是单独对时间处理，与对流量\上\下\部温度设定进行处理，并未结合时间与其他特征的关系，可以尝试自己构建合理的新特征。

5、 后处理

对已经给出的csv文件仍然可以分析其趋势，抓住评估的关键来调整文件内容使得结果更加精确。

6、 迭代步数
目前设置的迭代步数为200轮，其实这对于某些预测数据来说是不够的，可以尝试自己增大迭代步数

## 6.2 结语

总的来说，此次任务数据量不是很大，处理过程也是传统的机器学习赛题，适合小白入门学习与实践（适合俺）;

感谢Datawhale的敏姐，骆哥，鱼哥的无私分享以及aistudio平台的算力支持！！！

Datawhale是一个专注于AI与数据科学领域的开源组织，汇集了众多领域院校和知名企业的优秀学习者，聚合了一群有开源精神和探索精神的团队成员。Datawhale以"forthelearner,和学习者一起成长"为使命，鼓励真实地展现自我、开放包容、互信互助、敢于尝试和勇于担当。同时Datawhale 用开源的理念去探索开源内容、开源学习和开源方案，赋能人才培养，助力人才成长，建立起人与人，人与知识，人与企业和人与未来的联结。

欢迎各位同学一起来打榜挑战！！！

参考文献:

- 参考来源Datawhale社区:https://datawhaler.feishu.cn/docx/DGmZd217PoYhgexr2WPc6rObnCe